# Benchmark

This notebook provides a straightforward way to compare the PyZX optimization routines against other approaches on a standard set of benchmark circuits.

First we execute the standard set of imports:

In [1]:
import random, math, os, time
import sys; sys.path.append('..')
import pyzx as zx
%config InlineBackend.figure_format = 'svg'

The following class is some boilerplate around the simplification routines so that we can more easily print the relevant metrics

In [2]:
class CircuitComparer:
    def __init__(self, dirname, before, after):
        self.fname_before = os.path.join(dirname, before)
        if after:
            self.fname_after = os.path.join(dirname, after)
        else:
            self.fname_after = ""
        self.fname_tpar = ""
        if before.find('before') != -1:
            self.name = before[:-7]
        else:
            self.name = before
        self.has_run = False
    def __str__(self):
        return "CircuitComparer({}, {})".format(self.name, str(self.has_run))
    def __repr__(self):
        return str(self)
    
    def run(self):
        if self.has_run: return True
        if self.fname_after:
            c = zx.Circuit.from_quipper_file(self.fname_after).to_basic_gates()
            self.t_opt = c.tcount()
        else:
            self.t_opt = '-'
        c = zx.Circuit.load(self.fname_before).to_basic_gates()
        self.qubits = c.qubits
        if self.fname_tpar:
            c2 = zx.Circuit.load(self.fname_tpar)
            self.tpar = c2.tcount()
        else: self.tpar = "-"
        self.gatecount = len(c.gates)
        self.t_before = c.tcount()
        g = c.to_graph()
        t = time.time()
        zx.simplify.full_reduce(g)
        self.t_after = zx.tcount(g)
        self.time_simpl = time.time() - t
        t = time.time()
        self.extracts = True
        try: 
            c2 = zx.extract.streaming_extract(g,quiet=True)
            self.time_extr = time.time() - t
        except Exception:
            self.extracts = False
            self.time_extr = -1
        self.has_run = True
        return True
    
    def get_output(self):
        if not self.has_run:
            self.run()
        s = self.name.ljust(20) + str(self.qubits).rjust(7)
        s += str(self.gatecount).rjust(8) + str(self.t_before).rjust(9) + str(self.t_opt).rjust(7) 
        s += str(self.tpar).rjust(8) + str(self.t_after).rjust(8)
        s += "{:.2f}".format(self.time_simpl).rjust(11)
        s += "{:.2f}".format(self.time_extr).rjust(12)
        return s

Next we define a function that loads in a directory of circuit files. Note that the directory we target has up to 3 versions of each circuit:

* circuit_before   - This is the original circuit with any modifications, taken from the [Github page](https://github.com/njross/optimizer) of [[1]](#NRSCM)
* circuit_after    - This is the circuit produced by the optimization routines of [[1]](#NRSCM).
* circuit_tpar.qc  - This is the circuit produced by the Tpar algorithm [[2]](#Tpar).
  
<a id="NRSCM"></a>
[1] [Nam, Ross, Su, Childs, Maslov - Automated optimization of large quantum circuits with continuous parameters](https://www.nature.com/articles/s41534-018-0072-4)

<a id="Tpar"></a>
[2] [Amy, Maslov, Mosca - Polynomial-time T-depth Optimization of Clifford+T circuits via Matroid Partitioning](https://arxiv.org/abs/1303.2042)

In [3]:
def load_circuits(directory):
    d = directory
    beforefiles = []
    afterfiles = []
    tparfiles = []
    for f in os.listdir(d):
        if not os.path.isfile(os.path.join(d,f)): continue
        if f.find('before') != -1:
            beforefiles.append((f,d))
        elif f.find('tpar') != -1:
            tparfiles.append((f,d))
        elif f.find('.qc') != -1 or f.find('.tfc') != -1:
            beforefiles.append((f,d))
        else: afterfiles.append((f,d))
    
    circuits = []
    for f, d in beforefiles:
        if f.find('before') == -1:
            n = os.path.splitext(f)[0]
        else: n = f[:-7]
        for f2,d2 in afterfiles:
            if d!=d2: continue
            if f2.startswith(n):
                c = CircuitComparer(d, f, f2)
                circuits.append(c)
                break
        else:
            c = CircuitComparer(d, f, '')
            circuits.append(c)
        for f2,d2 in tparfiles:
            if d!=d2: continue
            if f2.startswith(n):
                circuits[-1].fname_tpar = os.path.join(d2,f2)
    
    return circuits

dir_fast_circuits = os.path.join('..', 'circuits', 'Fast')
fast_circuits = load_circuits(dir_fast_circuits)

The directory we target contains a subset of all benchmark circuits, chosen for given quick results. The following cell giving benchmark results of these circuits should therefore only take a few seconds to run. For the benchmarks of slower circuits see [below](#slowbench).
The columns have the following meaning:

* `Circuit     ` - The name of the circuit
* `qubits      ` - Amount of qubits in the circuit
* `G-count     ` - Gate count of original circuit
* `T-before    ` - Amount of T-gates in original circuit
* `T-NRSCM     ` - Amount of T-gates in optimised circuit of [[1]](#NRSCM)
* `T-par       ` - Amount of T-gates in optimised circuit of [[2]](#Tpar)
* `T-PyZX      ` - Amount of T-gates in optimised circuit made by PyZX
* `Time-Simp   ` - The time taken for running the simplification routine on the circuit
* `Time-Extract` - The time taken for extracting the circuit after the simplification

Note that not all circuits were present in the papers [[1]](#NRSCM) and [[2]](#Tpar) in which case the relevant columns are empty.

In [4]:
print("Circuit".ljust(20), "qubits", "G-count", "T-before", "T-NRSCM", " T-par", " T-PyZX", " Time-Simp", " Time-Extract")
for c in fast_circuits:
    print(c.get_output())

Circuit              qubits G-count T-before T-NRSCM  T-par  T-PyZX  Time-Simp  Time-Extract
Adder8                   23     637      266     56       -      56       0.22        0.07
adder_8                  24    1014      399    215     215     173       0.57        2.17
barenco_tof_10           19     514      224    100     100     100       0.14        0.15
barenco_tof_3             5      66       28     16      16      16       0.01        0.00
barenco_tof_4             7     130       56     28      28      28       0.03        0.00
barenco_tof_5             9     194       84     40      40      40       0.05        0.01
csla_mux_3_original      15     190       70     64       -      62       0.04        0.01
csum_mux_9_corrected     30     448      196     84       -      84       0.11        0.03
gf2^10_mult              30    1709      700    410     410     410       0.58        0.02
gf2^4_mult               12     275      112     68      68      68       0.08        0.

<a id="slowbench"></a>
And now we do the benchmark on the slower circuits. Note that this can take up to half an hour to complete.

In [12]:
dir_slow_circuits = os.path.join('..', 'circuits', 'Slow')
slow_circuits = load_circuits(dir_slow_circuits)
print("Circuit".ljust(20), "qubits", "G-count", "T-before", "T-NRSCM", " T-par", " T-PyZX", " Time-Simp", " Time-Extract")
for c in slow_circuits:
    if c.run():
        print(c.pretty())

Circuit              qubits G-count T-before T-NRSCM  T-par  T-PyZX  Time-Simp  Time-Extract
Adder16                  47    1437      602    120       -     120       0.97       12.62
Adder32                  95    3037     1274    248       -     248       2.06       66.42
Adder64                 191    6237     2618    504       -     504       5.24      202.47
cycle_17_3.qc            35   11512     4739      -       -    1797     543.72      293.03
gf2^16_mult              48    4397     1792   1040    1040    1040      12.47        0.59
ham15-high.qc            20    6010     2457      -       -    1021     106.21       19.00
ham15-med.qc             17    1436      574      -       -     212       3.97        2.07
mod_adder_1024           28    4855     1995   1011    1011    1011      45.87       59.85
QFT32                    32    1562      918    368       -     368       0.86       16.59
QFTAdd16                 32    1822     1026    402       -     402       1.12       21.